In [1]:
import mediapy as media
import copy
import mujoco
import numpy as np
from robot_descriptions.loaders.mujoco import load_robot_description

from mujoco_sysid.parameters import get_dynamic_parameters
from mujoco_sysid.regressors import joint_torque_regressor
from utils import ActuatorMotor, update_actuator
from mujoco_logger import SimLogger

model = load_robot_description("panda_mj_description", variant="panda_nohand")
# disable collisions
model.opt.disableflags |= (
    mujoco.mjtDisableBit.mjDSBL_CONTACT | mujoco.mjtDisableBit.mjDSBL_FRICTIONLOSS | mujoco.mjtDisableBit.mjDSBL_LIMIT
)

for actuator_id in range(model.nu):
    actuator = ActuatorMotor()
    update_actuator(model, actuator_id, actuator)

data = mujoco.MjData(model)

In [2]:
# model and data specifically to perform controller computations
cmodel = copy.deepcopy(model)
cdata = copy.deepcopy(data)


def controller(q, v, dv_desired, theta_est):
    """
    Inverse dynamics controller with feedforward term.
    """

    cdata.qpos[:] = q.copy()
    cdata.qvel[:] = v.copy()
    cdata.qacc[:] = dv_desired.copy()
    mujoco.mj_inverse(cmodel, cdata)
    mujoco.mj_rnePostConstraint(cmodel, cdata)

    Y = joint_torque_regressor(cmodel, cdata)
    return Y @ theta_est

In [3]:
# ideal set of dynamic parameters
theta = np.concatenate([get_dynamic_parameters(model, i) for i in model.jnt_bodyid])

In [4]:
from mujoco_logger import SimLog

log = SimLog("invdyn.json")


def construct_regression(log: SimLog):
    N = len(log)
    A = np.zeros((N * model.nv, len(theta)))
    b = np.zeros((N * model.nv,))

    for i in range(N):
        q = log[i].data("qpos")
        v = log[i].data("qvel")
        dv = log[i].data("qacc")
        # tau = log[i].data("qfrc_inverse")

        cdata.qpos[:] = q
        cdata.qvel[:] = v
        cdata.qacc[:] = dv

        mujoco.mj_inverse(cmodel, cdata)
        mujoco.mj_rnePostConstraint(cmodel, cdata)

        Y = joint_torque_regressor(cmodel, cdata)

        A[i * model.nv : (i + 1) * model.nv, :] = Y
        b[i * model.nv : (i + 1) * model.nv] = cdata.qfrc_inverse.copy()

    return A, b


A, b = construct_regression(log)

theta_est = np.linalg.lstsq(A, b, rcond=None)[0]

print("True parameters:")
print(theta)

print("Estimated parameters:")
print(theta_est)

np.linalg.norm(theta - theta_est)

True parameters:
[ 4.97068400e+00  1.92614005e-02  1.03439934e-02 -2.36703972e-01
  7.14663369e-01 -1.79087434e-04  7.17956456e-01  7.68922777e-03
  1.96620379e-02  9.21318890e-03  6.46926000e-01 -2.03199457e-03
 -1.85797147e-02  2.26100637e-03  8.50351116e-03 -3.98335872e-03
  2.81242848e-02  1.02611015e-02  7.68935939e-04  2.65349923e-02
  3.22860400e+00  8.88447249e-02  1.26729164e-01 -2.14708623e-01
  5.64949260e-02 -8.24833314e-03  5.28783820e-02 -5.48764810e-03
 -4.37725712e-03  1.82492023e-02  3.58789500e+00 -1.90768377e-01
  3.74644408e-01  9.85020693e-02  6.76772726e-02  2.77158413e-02
  3.23994297e-02  3.90535528e-03 -1.64448587e-03  7.75861474e-02
  1.22594600e+00 -1.46537325e-02  5.03434725e-02 -4.71216864e-02
  3.94275710e-02 -1.51524448e-03  3.14603723e-02 -4.60024554e-03
  2.16405199e-03  1.08695108e-02  1.66655500e+00  1.00241617e-01
 -2.35267569e-02 -1.75271589e-02  2.48046033e-03  1.52411091e-03
  1.05677661e-02 -1.03758875e-04  9.35690967e-05  1.17945603e-02
  7.3552

7.049623006039608

In [5]:
import jax
import jax.numpy as jnp
from mujoco_sysid.convert import theta2logchol


def logchol2theta(log_cholesky: jnp.ndarray) -> jnp.ndarray:
    alpha, d1, d2, d3, s12, s23, s13, t1, t2, t3 = log_cholesky

    exp_d1 = jnp.exp(d1)
    exp_d2 = jnp.exp(d2)
    exp_d3 = jnp.exp(d3)

    theta = jnp.array(
        [
            1,
            t1,
            t2,
            t3,
            s23**2 + t2**2 + t3**2 + exp_d2**2 + exp_d3**2,
            -s12 * exp_d2 - s13 * s23 - t1 * t2,
            s12**2 + s13**2 + t1**2 + t3**2 + exp_d1**2 + exp_d3**2,
            -s13 * exp_d3 - t1 * t3,
            -s23 * exp_d3 - t2 * t3,
            s12**2 + s13**2 + s23**2 + t1**2 + t2**2 + exp_d1**2 + exp_d2**2,
        ]
    )

    exp_2_alpha = jnp.exp(2 * alpha)
    theta *= exp_2_alpha

    return theta


logcholideal = np.concatenate([theta2logchol(thetai) for thetai in np.split(theta, 7)])

In [6]:
theta_ideal = jnp.concatenate([logchol2theta(logcholideal[i : i + 10]) for i in range(0, 70, 10)])

np.linalg.norm(theta - theta_ideal)

2024-07-05 11:55:50.581182: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


1.5977166e-07

In [7]:
A = A[: 1000 * model.nv, :]
b = b[: 1000 * model.nv]

In [8]:
np.linalg.norm(A @ theta_ideal - b)

9.89768e-05

In [9]:
import optax


# @jax.jit
# def loss_fn(theta, A, b):
#     # return jnp.linalg.norm(A @ theta - b)
#     return jnp.sum(optax.l2_loss(A @ theta, b))


@jax.jit
def all_logchol2theta(logchol):
    return jnp.concatenate([logchol2theta(logchol[i : i + 10]) for i in range(0, 70, 10)])


@jax.jit
def loss_fn_logchol(logchol, A, b):
    theta = all_logchol2theta(logchol)
    return jnp.mean(optax.l2_loss(A @ theta, b)) + 1000 * jnp.mean(optax.l2_loss(logchol, logcholideal))
    # return loss_fn(theta, A, b)


@jax.jit
def update(params, grads, opt_state):
    updates, new_opt_state = optimizer.update(grads, opt_state)
    new_params = optax.apply_updates(params, updates)
    return new_params, new_opt_state, updates


start_learning_rate = 2
optimizer = optax.adagrad(start_learning_rate)
# params = logcholideal
params = jnp.zeros(70)
opt_state = optimizer.init(params)

for i in range(6000):
    grads = jax.grad(loss_fn_logchol)(params, A, b)
    params, opt_state, updates = update(params, grads, opt_state)
    print(i, np.linalg.norm(theta_ideal - all_logchol2theta(params)))

theta_est = params

0 7.43234
1 7.343218
2 7.2973332
3 7.2234063
4 7.1185617
5 6.981863
6 6.8195496
7 6.638346
8 6.448657
9 6.259862
10 6.079131
11 5.909893
12 5.7991433
13 5.6044717
14 5.520382
15 5.4776464
16 5.612284
17 5.977417
18 15.1389065
19 7.2644076
20 7.2484937
21 7.2418637
22 7.2377615
23 7.2466526
24 7.338172
25 7.788464
26 7.351474
27 9.511699
28 7.200223
29 7.174031
30 7.1638675
31 7.162124
32 7.1712976
33 7.1793947
34 7.171129
35 7.1561613
36 7.1409144
37 7.1270275
38 7.101019
39 7.1063657
40 6.9928074
41 7.3840303
42 7.001
43 6.920773
44 6.9625573
45 6.9692693
46 7.0272813
47 6.9508286
48 7.196608
49 7.0190306
50 7.481346
51 7.0050054
52 7.596371
53 6.929977
54 7.5740275
55 7.132374
56 6.9501805
57 7.2071495
58 7.627421
59 7.6087394
60 6.9065266
61 6.7363405
62 6.8663707
63 6.8791766
64 7.0960975
65 6.8139973
66 7.1290326
67 6.702626
68 7.268919
69 7.0092807
70 6.377799
71 6.5196643
72 6.4832215
73 6.65581
74 6.4615827
75 7.4628224
76 6.1287694
77 6.6019683
78 6.148769
79 6.5109253
80 6.04

In [10]:
np.linalg.norm(theta_ideal - all_logchol2theta(theta_est)), np.linalg.norm(A @ all_logchol2theta(theta_est) - b)

(0.023447527, 20.892828)